In [1]:
import pandas as pd
import geopandas as gpd

from core.utils import solve_path

In [2]:
from core.transform_data import TransformarIndicadores

In [3]:
df = pd.read_csv('data/csv_data/ObservaSampaDadosAbertosIndicadoresCSV.csv',
           encoding='utf-8', sep=';')

In [4]:
distorcao_idade_ano = [
    '04.01.03 Taxa de distorção da idade-ano para o ano no Ensino Fundamental nos anos iniciais (%)',
   '04.01.04 Taxa de distorção da idade-ano para o ano no Ensino Fundamental nos anos finais (%)',
]

In [5]:
raca_alunos = [
    'Alunos da rede municipal de ensino da raça/cor amarela (%)',
    'Alunos da rede municipal de ensino da raça/cor branca (%)',
    'Alunos da rede municipal de ensino da raça/cor indígena (%)',
    'Alunos da rede municipal de ensino da raça/cor parda (%)',
    'Alunos da rede municipal de ensino da raça/cor preta (%)',
       
       ]

In [6]:
abandono = [
    'Taxa de abandono escolar no Ensino Fundamental da rede municipal (%)',
   'Taxa de abandono escolar no Ensino Médio da rede municipal (%)',
]

In [7]:
def filtrar_distritos(df):
    
    mask = df['Região'].str.endswith('(Distrito)')
    
    return df[mask].copy().reset_index(drop=True)

In [8]:
indis_interesse = []
indis_interesse.extend(distorcao_idade_ano)
indis_interesse.extend(raca_alunos)
indis_interesse.extend(abandono)


In [9]:
interesse = df[df['Nome'].isin(indis_interesse)].reset_index(drop=True)

In [10]:
interesse = filtrar_distritos(interesse)

In [11]:
interesse

,Nome,Região,Período,Resultado
0,04.01.03 Taxa de distorção da idade-ano para o...,Água Rasa (Distrito),2016,"5,0107"
1,04.01.03 Taxa de distorção da idade-ano para o...,Água Rasa (Distrito),2018,"6,4843"
2,04.01.03 Taxa de distorção da idade-ano para o...,Água Rasa (Distrito),2019,"5,6323"
3,04.01.03 Taxa de distorção da idade-ano para o...,Água Rasa (Distrito),2017,"6,8834"
4,04.01.03 Taxa de distorção da idade-ano para o...,Água Rasa (Distrito),2020,"3,5599"
...,...,...,...,...
4263,Taxa de abandono escolar no Ensino Médio da re...,Sapopemba (Distrito),2008,"4,2328"
4264,Taxa de abandono escolar no Ensino Médio da re...,Sapopemba (Distrito),2013,"17,3913"
4265,Taxa de abandono escolar no Ensino Médio da re...,Sapopemba (Distrito),2018,"10,2564"
4266,Taxa de abandono escolar no Ensino Médio da re...,Sapopemba (Distrito),2017,"4,7458"


In [12]:
interesse.groupby('Nome').count()

,Região,Período,Resultado
Nome,,,
04.01.03 Taxa de distorção da idade-ano para o ano no Ensino Fundamental nos anos iniciais (%),425,425,425
Alunos da rede municipal de ensino da raça/cor amarela (%),558,558,558
Alunos da rede municipal de ensino da raça/cor branca (%),558,558,558
Alunos da rede municipal de ensino da raça/cor indígena (%),443,443,443
Alunos da rede municipal de ensino da raça/cor parda (%),558,558,558
Alunos da rede municipal de ensino da raça/cor preta (%),558,558,558
Taxa de abandono escolar no Ensino Fundamental da rede municipal (%),1084,1084,1084
Taxa de abandono escolar no Ensino Médio da rede municipal (%),84,84,84


In [13]:
def remover_acentos(name):
    
    acento_letra = {
        'ç' : 'c',
        'á' : 'a',
        'â' : 'a',
        'à' : 'a',
        'ã' : 'a',
        'ä' : 'a',
        'é' : 'e',
        'ê' : 'e',
        'è' : 'e',
        'ë' : 'e',
        'í' : 'i',
        'î' : 'i',
        'ì' : 'i',
        'ï' : 'i',
        'ó' : 'o',
        'ô' : 'o',
        'ò' : 'o',
        'ø' : 'o',
        'õ' : 'o',
        'ö' : 'o',
        'ú' : 'u',
        'û' : 'u',
        'ù' : 'u',
        'ü' : 'u',
        'ñ' : 'n',
        'ý' : 'y'
    }
    
    chars = list(name)
    
    return ''.join([acento_letra.get(char, char) for char in chars])

In [14]:
def clean_distrito_name(name):
    
    lowered = name.lower().replace('(distrito)', '').strip()
    sem_acento = remover_acentos(lowered)
    
    return sem_acento.upper()

In [15]:
interesse['ds_nome'] = interesse['Região'].apply(clean_distrito_name)

In [16]:
folder_maps = solve_path('maps', parent='static_files')

In [17]:
folder_distritos = solve_path('distritos', parent=folder_maps)

In [18]:
shp_distritos = solve_path('SIRGAS_SHP_distrito_polygon.shp', parent=folder_distritos)

In [19]:
distritos= gpd.read_file(shp_distritos)

In [25]:
merged = pd.merge(interesse, distritos, on='ds_nome', how='left')

In [21]:
merged['geometry'].isnull().any()

False

In [22]:
type(merged)

pandas.core.frame.DataFrame

In [23]:
municipio_todo = distritos.dissolve()

In [27]:
distritos.set_crs(epsg='3198')

,ds_codigo,ds_subpref,ds_cd_sub,ds_sigla,ds_nome,geometry
0,63,PIRITUBA-JARAGUA,02,PIR,PIRITUBA,"POLYGON ((325109.668 7405122.828, 325106.786 7..."
1,95,PIRITUBA-JARAGUA,02,SDO,SAO DOMINGOS,"POLYGON ((319854.036 7402894.086, 319864.990 7..."
2,42,PIRITUBA-JARAGUA,02,JAR,JARAGUA,"POLYGON ((319172.403 7405179.204, 319174.204 7..."
3,11,FREGUESIA-BRASILANDIA,03,BRL,BRASILANDIA,"POLYGON ((324895.091 7404257.969, 324893.588 7..."
4,29,FREGUESIA-BRASILANDIA,03,FRE,FREGUESIA DO O,"POLYGON ((327469.803 7399113.522, 327449.180 7..."
...,...,...,...,...,...,...
91,44,SAO MIGUEL,23,JDH,JARDIM HELENA,"POLYGON ((352915.621 7402621.049, 352944.248 7..."
92,24,ITAQUERA,27,CLD,CIDADE LIDER,"POLYGON ((345181.289 7394204.666, 345180.470 7..."
93,57,ITAQUERA,27,PQC,PARQUE DO CARMO,"POLYGON ((352597.619 7390119.444, 352588.233 7..."
94,47,ITAQUERA,27,JBO,JOSE BONIFACIO,"POLYGON ((354091.203 7396702.971, 354091.662 7..."


In [1]:
print('qa')

qa
